## Import Libraries

In [1]:
import os
import pickle
import time
import langchain
from langchain import OpenAI
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.chains.qa_with_sources.loading import load_qa_with_sources_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import UnstructuredURLLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from configparser import ConfigParser

## Initialize LLM

In [4]:
config = ConfigParser()
config.read('../config/config.ini')

['../config/config.ini']

In [5]:
api_key = config['OPEN_AI']['api_key']

In [6]:
llm = OpenAI(temperature=0.9, max_tokens=500, openai_api_key=api_key)

/tmp/ipykernel_86898/251989380.py:1: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm = OpenAI(temperature=0.9, max_tokens=500, openai_api_key=api_key)


## Explore on Sample Data

In [7]:
loaders = UnstructuredURLLoader(urls=[
    'https://finance.yahoo.com/news/meta-platforms-meta-dips-more-224522829.html',
    'https://finance.yahoo.com/news/jpmorgan-raises-meta-platforms-inc-031043241.html',
    'https://finance.yahoo.com/news/meta-platforms-inc-meta-among-131432947.html',
])
data = loaders.load() 
len(data)

3

In [8]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 100)
docs = text_splitter.split_documents(data)

In [9]:
len(docs)

27

### Create Embeddings

In [ ]:
embeddings = OpenAIEmbeddings(openai_api_key = api_key)
vector_idx = FAISS.from_documents(docs, embeddings)

In [12]:
VECTOR_INDEX_PATH = 'model_artifacts/vector_index'

In [ ]:
with open(VECTOR_INDEX_PATH, 'wb') as f:
    pickle.dump(vector_idx ,f)

### Retrieval QA

In [ ]:
langchain.debug=True

chain = RetrievalQAWithSourcesChain.from_llm(llm=llm, retriever=vectorIndex.as_retriever())
chain

In [ ]:
query = "What is Meta's stock performance in the latest reports?"
chain({"question": query}, return_only_outputs=True)